# Module biogeme.loglikelihood 

## Examples of use of each function

This webpage is for programmers who need examples of use of the functions of the module. The examples are designed to illustrate the syntax. They do not correspond to any meaningful model. For examples of models, visit  [biogeme.epfl.ch](http://biogeme.epfl.ch).

In [1]:
import datetime
print(datetime.datetime.now())

In [2]:
import biogeme.version as ver
print(ver.getText())

In [3]:
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.loglikelihood as ll
import biogeme.expressions as ex
import biogeme.models as md

This module provides some basic expressions for the contribution of an observation to the (log) likelihood function.

Let's consider first a simple choice model.

In [4]:
V1 = 0
beta = ex.Beta('beta', 0, None, None, 0)
sigma = ex.Beta('sigma', 1, 0, None, 0)
V2 = beta + sigma * ex.bioDraws('V2', 'NORMAL')
V = {1: V1, 2: V2}
condprob = md.logit(V, None, 0)
prob = ex.MonteCarlo(condprob)
print(prob)

MonteCarlo(exp(_bioLogLogitFullChoiceSet(1:`0`, 2:(beta(0) + (sigma(1) * bioDraws("V2", "NORMAL"))))))


The first function simply takes the log of the probability for each observation.

In [5]:
loglike = ll.loglikelihood(prob)
print(loglike)

log(MonteCarlo(exp(_bioLogLogitFullChoiceSet(1:`0`, 2:(beta(0) + (sigma(1) * bioDraws("V2", "NORMAL")))))))


The second function also involves the integral using Monte-Carlo simulation.

In [6]:
loglike = ll.mixedloglikelihood(condprob)
print(loglike)

log(MonteCarlo(exp(_bioLogLogitFullChoiceSet(1:`0`, 2:(beta(0) + (sigma(1) * bioDraws("V2", "NORMAL")))))))


Regression models are often used in the context of hybrid choice models. Consider the following model.

In [7]:
x = ex.Variable('x')
y = ex.Variable('y')
beta = ex.Beta('beta', 1, None, None, 0)
sigma = ex.Beta('sigma', 1, None, None, 0)
intercept = ex.Beta('intercept', 0, None, None, 0)
model = intercept + beta * x

The following function calculates the contribution to the likelihood. It is 

 \\[\frac{1}{\sigma} \phi\left( \frac{y-m}{\sigma} \right),\\]
 
 where $\phi(\cdot)$ is the pdf of the normal distribution.



In [8]:
like = ll.likelihoodregression(y, model, sigma)
print(like)

exp(((((-(((y - (intercept(0) + (beta(1) * x))) / sigma(1)) ** `2`)) / `2`) - log(sigma(1))) - `0.9189385332`))


The following function calculates the log of the contribution to the likelihood. It is

\\[-\left( \frac{(y-m)^2}{2\sigma^2} \right) -
              \log(\sigma) - \frac{1}{2}\log(2\pi).\\]

In [9]:
loglike = ll.loglikelihoodregression(y, model, sigma)
print(loglike)

((((-(((y - (intercept(0) + (beta(1) * x))) / sigma(1)) ** `2`)) / `2`) - log(sigma(1))) - `0.9189385332`)


We compare the two on a small database

In [10]:
df = pd.DataFrame({'x': [-2, -1, 0, 1, 2],
                   'y': [1, 1, 1, 1, 1]})
myData = db.Database('test', df)

In [11]:
lr = like.getValue_c(myData)
lr

[0.004431848481226311,
 0.05399096735729239,
 0.24197072830215655,
 0.398942286638567,
 0.24197072830215655]

In [12]:
np.log(lr)

array([-5.41893852, -2.91893852, -1.41893852, -0.91893852, -1.41893852])

In [13]:
loglike.getValue_c(myData)

[-5.418938517570496,
 -2.9189385175704956,
 -1.4189385175704956,
 -0.9189385175704956,
 -1.4189385175704956]